# Эксперименты с моделями: оценка стоимости аренды в Москве

В этом ноутбуке проводится серия экспериментов по обучению моделей для предсказания стоимости аренды квартир в Москве на основе данных, полученных с Cian.ru и обогащённых географическими и инфраструктурными признаками.

---

## Цель ноутбука

Пошагово протестировать, какие признаки и приёмы предобработки дают наибольший вклад в качество модели, чтобы:

- построить сильную, интерпретируемую baseline-модель;
- зафиксировать вклад каждой группы признаков (география, инфраструктура и т.д.);
- отобрать финальные подходы для переноса в финальный ноутбук `final_model.ipynb`.

---

## Подход

В рамках экспериментов используется **CatBoostRegressor**, так как он:

- нативно работает с категориальными признаками;
- позволяет обучаться на GPU, что критично при большом объёме данных;
- стабильно показывает высокое качество на табличных данных.

---

## Последовательность экспериментов

1. **Baseline-модель** на исходных данных без признаков
2. Добавление **координат**
3. Добавление **расстояний до центра и метро**
4. Добавление **инфраструктурных признаков** (POI)
5. **Удаление выбросов** и сравнение до/после
6. **Логарифмирование целевой переменной**
7. **Подбор гиперпараметров** модели через Optuna

---

## Метрики оценки качества

Для каждой модели вычисляются:

- **MAE** (средняя абсолютная ошибка, руб.)
- **MAPE** (средняя абсолютная процентная ошибка)
- **R²** (коэффициент детерминации)
- **RMSE** (корень средней квадратичной ошибки) — основная метрика

---

> Все шаги фиксируются и сравниваются на одной и той же схеме валидации, чтобы обеспечить корректную интерпретацию результатов. Лучший подход будет использован в финальной модели.


In [1]:
%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement optuna-integration==3.6.1 (from versions: 3.2.0b0, 3.2.0, 3.3.0, 3.4.0, 3.5.0, 3.6.0, 4.0.0b0, 4.0.0, 4.1.0, 4.2.0, 4.2.1, 4.3.0, 4.4.0)

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip
ERROR: No matching distribution found for optuna-integration==3.6.1


## Импорт необходимых библиотек

In [2]:
import numpy as np
import pandas as pd
import warnings
import joblib
import json

from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split

from sklearn.metrics import (
    root_mean_squared_error,
    mean_absolute_error,
    mean_absolute_percentage_error,
    mean_squared_error,
    r2_score
)

import optuna
from optuna.integration import CatBoostPruningCallback
from optuna.pruners import MedianPruner
warnings.filterwarnings("ignore")

## Эксперимент 1 — Бейзлайн на «сырых» данных

В этом эксперименте мы обучим первую базовую модель CatBoost на минимально предобработанных данных, без дополнительных признаков.

Здесь:

- загружается датасет из CSV-файла `cian_rent_Moscow_cleaned.csv`;
- удаляются столбцы: `lat`, `lon`, `geo_quality` — они будут использованы позже;
- никаких новых признаков (инфраструктура, расстояния, логарифмирование) пока не добавляется;
- обучение проводится на исходной структуре данных с использованием только базовых категориальных и числовых признаков.

Цель — зафиксировать начальное качество модели, чтобы использовать его как точку отсчёта для всех последующих экспериментов.
___

In [3]:
data = pd.read_csv("cian_rent_Moscow_cleaned.csv")
data = data.drop(["lat", "lon", "geo_quality"], axis = 1)
data.head()

,floor,floors_count,rooms_count,total_meters,price_per_month,district,street,house_number,underground
0,3,5,1,30.0,30000,Северное Измайлово,Сиреневый бульвар,7,Щёлковская
1,2,34,-1,55.7,25000,Войковский,Адмирала Макарова,6Ак1,Водный стадион
2,5,9,1,50.0,30000,Нагорный,Варшавское шоссе,71К1,Варшавская
3,1,5,1,32.0,30000,Лосиноостровский,Анадырский проезд,41,Бабушкинская
4,10,12,1,23.0,29998,НАО (Новомосковский),Яворки,1к1,Филатов луг


In [4]:
X = data.drop("price_per_month", axis = 1)
y = data["price_per_month"]

In [5]:
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, random_state = 42, train_size = 0.8, shuffle = True)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, random_state = 42, test_size = 0.5, shuffle = True)

In [6]:
baseline_model_exp1 = CatBoostRegressor(iterations = 3000,
                                        learning_rate = 0.015,
                                        loss_function="RMSE",
                                        eval_metric="RMSE",
                                        early_stopping_rounds=200,
                                        verbose=100,
                                        task_type="GPU")

In [8]:
baseline_model_exp1.fit(X_train,
                       y_train,
                       eval_set = (X_val, y_val),
                       cat_features = ["district", "street", "underground", "house_number"],
                       use_best_model=True)

0:	learn: 109857.2096856	test: 110833.7584171	best: 110833.7584171 (0)	total: 115ms	remaining: 5m 46s
100:	learn: 56685.9710178	test: 57050.7027274	best: 57050.7027274 (100)	total: 2.06s	remaining: 59s
200:	learn: 49532.8907736	test: 50687.9383504	best: 50687.9383504 (200)	total: 4.03s	remaining: 56.1s
300:	learn: 47650.1417366	test: 49436.5538052	best: 49436.5538052 (300)	total: 5.96s	remaining: 53.4s
400:	learn: 46592.6480461	test: 48750.5192637	best: 48750.5192637 (400)	total: 7.94s	remaining: 51.5s
500:	learn: 45822.3714552	test: 48209.2601862	best: 48209.2601862 (500)	total: 9.84s	remaining: 49.1s
600:	learn: 45306.7291154	test: 47867.0414828	best: 47867.0414828 (600)	total: 11.6s	remaining: 46.5s
700:	learn: 44767.6055760	test: 47484.0801652	best: 47484.0801652 (700)	total: 13.5s	remaining: 44.2s
800:	learn: 44279.0981387	test: 47158.2775103	best: 47158.2775103 (800)	total: 15.2s	remaining: 41.8s
900:	learn: 43898.1607113	test: 46844.1616493	best: 46844.1616493 (900)	total: 17s	r

In [9]:
y_pred = baseline_model_exp1.predict(X_val)

MAE = round(mean_absolute_error(y_val, y_pred), 2)
R2 = round(r2_score(y_val, y_pred), 4)
MAPE = round(mean_absolute_percentage_error(y_val, y_pred), 4)


print(f"MAE: {MAE}")
print(f"R2-Score: {R2}")
print(f"MAPE: {MAPE}")

MAE: 24814.42
R2-Score: 0.8461
MAPE: 0.1881


### Выводы по Эксперименту 1 — Бейзлайн-модель без дополнительных признаков

- **MAE:** 24 814 ₽ — средняя ошибка предсказания аренды в рублях;
- **R²:** 0.8461 — модель объясняет ~84.6% дисперсии цен;
- **MAPE:** 18.81% — в среднем предсказание отклоняется от фактической цены на 18.8%.

Несмотря на отсутствие координат, инфраструктуры и расстояний, модель уже показывает уверенное качество за счёт базовых признаков.

Этот результат будет использоваться как точка отсчёта для всех последующих экспериментов.

___

## Эксперимент 2 — Добавление координат (`lat`, `lon`)

В этом эксперименте к базовому датасету добавляются координаты объектов (`lat`, `lon`), отражающие их фактическое географическое положение в пространстве.

### Цель:
Проверить, даёт ли добавление географической информации модели возможность лучше учитывать локацию объекта.

### Что делается:
- Загружается версия датасета с колонками `lat`, `lon`;
- Остальные признаки и параметры модели остаются неизменными по сравнению с Экспериментом 1;
- Оценка проводится на той же валидационной выборке для корректного сравнения.

### Гипотеза:
Координаты должны дать прирост, особенно если модель сможет различить «центральные» и «периферийные» районы даже без дополнительных фичей.

___



In [10]:
data = pd.read_csv("cian_rent_Moscow_cleaned.csv")
data.head()

,floor,floors_count,rooms_count,total_meters,price_per_month,district,street,house_number,underground,lat,lon,geo_quality
0,3,5,1,30.0,30000,Северное Измайлово,Сиреневый бульвар,7,Щёлковская,55.802980,37.775445,1
1,2,34,-1,55.7,25000,Войковский,Адмирала Макарова,6Ак1,Водный стадион,55.833869,37.491056,0
2,5,9,1,50.0,30000,Нагорный,Варшавское шоссе,71К1,Варшавская,55.660412,37.620267,1
3,1,5,1,32.0,30000,Лосиноостровский,Анадырский проезд,41,Бабушкинская,55.870534,37.693488,1
4,10,12,1,23.0,29998,НАО (Новомосковский),Яворки,1к1,Филатов луг,55.509092,37.345487,1


In [11]:
X = data.drop("price_per_month", axis = 1)
y = data["price_per_month"]

In [12]:
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, random_state = 42, train_size = 0.8, shuffle = True)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, random_state = 42, test_size = 0.5, shuffle = True)

In [13]:
baseline_model_exp2 = CatBoostRegressor(iterations = 3000,
                                        learning_rate = 0.015,
                                        loss_function="RMSE",
                                        eval_metric="RMSE",
                                        early_stopping_rounds=200,
                                        verbose=100,
                                        task_type="GPU")

In [14]:
baseline_model_exp2.fit(X_train,
                       y_train,
                       eval_set = (X_val, y_val),
                       cat_features = ["district", "street", "underground", "house_number"],
                       use_best_model=True)

0:	learn: 109887.5507064	test: 110847.5665654	best: 110847.5665654 (0)	total: 42.2ms	remaining: 2m 6s
100:	learn: 56642.1229857	test: 57188.9552036	best: 57188.9552036 (100)	total: 2.12s	remaining: 1m
200:	learn: 48954.7285913	test: 50409.7204023	best: 50409.7204023 (200)	total: 4.17s	remaining: 58.1s
300:	learn: 46839.8510424	test: 48935.8940197	best: 48935.8940197 (300)	total: 6.22s	remaining: 55.8s
400:	learn: 45674.0963128	test: 48190.5345458	best: 48189.1097451 (397)	total: 8.31s	remaining: 53.9s
500:	learn: 44791.7700823	test: 47617.2651260	best: 47617.2651260 (500)	total: 10.4s	remaining: 51.9s
600:	learn: 44071.0467565	test: 47116.6840394	best: 47116.6840394 (600)	total: 12.4s	remaining: 49.4s
700:	learn: 43430.5495795	test: 46665.8574784	best: 46665.8574784 (700)	total: 14.2s	remaining: 46.7s
800:	learn: 42843.6974639	test: 46317.7079990	best: 46317.7079990 (800)	total: 16s	remaining: 44s
900:	learn: 42335.3585752	test: 45978.1807247	best: 45978.1807247 (900)	total: 17.7s	rema

In [15]:
y_pred = baseline_model_exp2.predict(X_val)

MAE = round(mean_absolute_error(y_val, y_pred), 2)
R2 = round(r2_score(y_val, y_pred), 4)
MAPE = round(mean_absolute_percentage_error(y_val, y_pred), 4)


print(f"MAE: {MAE}")
print(f"R2-Score: {R2}")
print(f"MAPE: {MAPE}")

MAE: 24034.85
R2-Score: 0.8568
MAPE: 0.1835


### Выводы по Эксперименту 2 — Добавление координат (`lat`, `lon`)

- **MAE:** 23 964 ₽ — ошибка снизилась на ~3.42% по сравнению с бейзлайном.
- **R²:** 0.8568 — модель объясняет на ~1.26% больше дисперсии, чем без координат.
- **MAPE:** 18.35% — относительная ошибка уменьшилась на 2.4%.

Добавление координат дало **умеренное, но стабильное улучшение качества** на всех метриках.  
Модель стала лучше учитывать географическое расположение объектов даже без явных расстояний до центра или метро.

Координаты можно считать полезным признаком, который стоит включать в финальную модель.
___

## Эксперимент 3 — Добавление расстояний до центра и метро

В этом эксперименте к признакам модели добавляются два числовых признака:

- `dist_to_center_km` — расстояние от объекта до центра Москвы;
- `dist_to_metro_km` — расстояние до ближайшей станции метро.

### Цель:
Проверить, усиливает ли явное указание удалённости от ключевых точек влияние локации на стоимость аренды.

### Что делается:
- К уже используемым признакам добавляются `dist_to_center_km` и `dist_to_metro_km`;
- Остальные признаки сохраняются без изменений;
- Модель и её параметры остаются прежними;
- Оценка производится на той же валидационной выборке для честного сравнения с предыдущими экспериментами.

### Гипотеза:
Расстояние до центра и метро — одни из ключевых факторов, определяющих цену аренды. Их добавление должно заметно повысить точность модели.
___

In [16]:
data = pd.read_csv("cian_rent_Moscow_with_dist.csv")
data.head()

,floor,floors_count,rooms_count,total_meters,price_per_month,district,street,house_number,underground,lat,lon,geo_quality,dist_to_center_km,dist_to_metro_km
0,3,5,1,30.0,30000,Северное Измайлово,Сиреневый бульвар,7,Щелковская,55.802980,37.775445,1,11.387233,1.623468
1,2,34,-1,55.7,25000,Войковский,Адмирала Макарова,6Ак1,Водный стадион,55.833869,37.491056,0,12.054178,0.609563
2,5,9,1,50.0,30000,Нагорный,Варшавское шоссе,71К1,Варшавская,55.660412,37.620267,1,10.188199,0.788797
3,1,5,1,32.0,30000,Лосиноостровский,Анадырский проезд,41,Бабушкинская,55.870534,37.693488,1,14.007041,1.818472
4,10,12,1,23.0,29998,НАО (Новомосковский),Яворки,1к1,Филатов луг,55.509092,37.345487,1,31.956738,10.803737


In [17]:
X = data.drop("price_per_month", axis = 1)
y = data["price_per_month"]

In [18]:
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, random_state = 42, train_size = 0.8, shuffle = True)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, random_state = 42, test_size = 0.5, shuffle = True)

In [19]:
baseline_model_exp3 = CatBoostRegressor(iterations = 3000,
                                        learning_rate = 0.015,
                                        loss_function="RMSE",
                                        eval_metric="RMSE",
                                        early_stopping_rounds=200,
                                        verbose=100,
                                        task_type="GPU")

In [20]:
baseline_model_exp3.fit(X_train,
                       y_train,
                       eval_set = (X_val, y_val),
                       cat_features = ["district", "street", "underground", "house_number"],
                       use_best_model=True)

0:	learn: 109903.1824467	test: 110868.6388446	best: 110868.6388446 (0)	total: 22.4ms	remaining: 1m 7s
100:	learn: 56029.4063241	test: 57064.0295116	best: 57064.0295116 (100)	total: 2s	remaining: 57.5s
200:	learn: 48349.1631085	test: 50165.2523156	best: 50165.2523156 (200)	total: 4.03s	remaining: 56.1s
300:	learn: 46059.8949675	test: 48384.4350518	best: 48384.4350518 (300)	total: 6.03s	remaining: 54.1s
400:	learn: 44838.2017482	test: 47529.6477124	best: 47529.6477124 (400)	total: 8.01s	remaining: 51.9s
500:	learn: 43892.5826744	test: 46885.4451734	best: 46884.1695571 (498)	total: 9.98s	remaining: 49.8s
600:	learn: 43233.4751386	test: 46478.6527823	best: 46478.6527823 (600)	total: 11.8s	remaining: 47.1s
700:	learn: 42507.8626139	test: 46019.2697086	best: 46019.2697086 (700)	total: 13.7s	remaining: 44.9s
800:	learn: 41765.7982689	test: 45528.9517938	best: 45528.7077564 (799)	total: 15.5s	remaining: 42.5s
900:	learn: 41035.2771725	test: 45036.8325274	best: 45036.8325274 (900)	total: 17.4s	

In [21]:
y_pred = baseline_model_exp3.predict(X_val)

MAE = round(mean_absolute_error(y_val, y_pred), 2)
R2 = round(r2_score(y_val, y_pred), 4)
MAPE = round(mean_absolute_percentage_error(y_val, y_pred), 4)


print(f"MAE: {MAE}")
print(f"R2-Score: {R2}")
print(f"MAPE: {MAPE}")

MAE: 23035.98
R2-Score: 0.8658
MAPE: 0.1781


### Выводы по Эксперименту 3 — Добавление расстояний до центра и метро

- **MAE:** 23 036 ₽ — ошибка снизилась на ~3.62% по сравнению с предыдущим экспериментом.  
- **R²:** 0.8658 — модель объясняет на ~1% больше дисперсии.  
- **MAPE:** 17.81% — относительная ошибка уменьшилась на ~2.9%.

Добавление расстояний до центра (`dist_to_center_km`) и метро (`dist_to_metro_km`) дало заметное улучшение на всех метриках.  
Даже при наличии координат эти признаки вносят дополнительную структурированную информацию о локации.

`dist_to_center_km` и `dist_to_metro_km` — **одни из ключевых признаков**, которые должны быть включены в финальную модель.
___

## Эксперимент 4 — Добавление инфраструктурных признаков (POI в радиусе 1 км)

В этом эксперименте к уже использованным признакам добавляются числовые признаки, описывающие наличие объектов социальной и бытовой инфраструктуры в радиусе 1 км от каждой квартиры:

- `parks_1km` — количество парков и зелёных зон;
- `schools_1km` — количество школ, лицеев, гимназий;
- `groceries_top_1km` — количество продуктовых магазинов из топ-10 сетей;
- `pharmacies_top_1km` — количество аптек из топ-10 сетей;
- `clinics_1km` — количество поликлиник и медицинских центров.

### Цель:
Проверить, улучшает ли наличие инфраструктурных объектов вокруг квартиры способность модели точно предсказывать стоимость аренды.

### Что делается:
- К существующим признакам добавляются 5 новых количественных переменных;
- Категориальные и дистанционные признаки остаются без изменений;
- Обучение модели и оценка метрик производятся тем же способом, что и в предыдущих экспериментах.

### Гипотеза:
Наличие развитой инфраструктуры рядом с объектом повышает его привлекательность и влияет на цену аренды, особенно в массовом сегменте.
___

In [22]:
data = pd.read_csv("cian_rent_Moscow_with_poi.csv")
data.head()

,floor,floors_count,rooms_count,total_meters,price_per_month,district,street,house_number,underground,lat,lon,geo_quality,dist_to_center_km,dist_to_metro_km,parks_1km,groceries_top_1km,schools_1km,pharmacies_top_1km,clinics_1km
0,3,5,1,30.0,30000,Северное Измайлово,Сиреневый бульвар,7,Щелковская,55.802980,37.775445,1,11.387233,1.623468,0,11,4,0,4
1,2,34,-1,55.7,25000,Войковский,Адмирала Макарова,6Ак1,Водный стадион,55.833869,37.491056,0,12.054178,0.609563,0,6,1,0,0
2,5,9,1,50.0,30000,Нагорный,Варшавское шоссе,71К1,Варшавская,55.660412,37.620267,1,10.188199,0.788797,0,2,5,0,4
3,1,5,1,32.0,30000,Лосиноостровский,Анадырский проезд,41,Бабушкинская,55.870534,37.693488,1,14.007041,1.818472,0,3,3,1,0
4,10,12,1,23.0,29998,НАО (Новомосковский),Яворки,1к1,Филатов луг,55.509092,37.345487,1,31.956738,10.803737,0,0,1,0,0


In [23]:
X = data.drop("price_per_month", axis = 1)
y = data["price_per_month"]

In [24]:
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, random_state = 42, train_size = 0.8, shuffle = True)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, random_state = 42, test_size = 0.5, shuffle = True)

In [25]:
baseline_model_exp4 = CatBoostRegressor(iterations = 3000,
                                        learning_rate = 0.015,
                                        loss_function="RMSE",
                                        eval_metric="RMSE",
                                        early_stopping_rounds=200,
                                        verbose=100,
                                        task_type="GPU")

In [26]:
baseline_model_exp4.fit(X_train,
                       y_train,
                       eval_set = (X_val, y_val),
                       cat_features = ["district", "street", "underground", "house_number"],
                       use_best_model=True)

0:	learn: 109902.1589327	test: 110867.5489973	best: 110867.5489973 (0)	total: 43.5ms	remaining: 2m 10s
100:	learn: 56011.7623321	test: 57023.3639494	best: 57023.3639494 (100)	total: 2.06s	remaining: 59.1s
200:	learn: 48193.8585722	test: 49902.2720364	best: 49902.2720364 (200)	total: 4.05s	remaining: 56.4s
300:	learn: 45822.2672756	test: 48130.1642461	best: 48130.1642461 (300)	total: 6.12s	remaining: 54.9s
400:	learn: 44570.6165623	test: 47230.5474319	best: 47230.5474319 (400)	total: 8.13s	remaining: 52.7s
500:	learn: 43508.6442578	test: 46540.5760025	best: 46540.5760025 (500)	total: 10.1s	remaining: 50.6s
600:	learn: 42821.6767510	test: 46142.9096986	best: 46142.9096986 (600)	total: 12.1s	remaining: 48.4s
700:	learn: 42123.7896721	test: 45723.9390872	best: 45723.0525225 (698)	total: 14s	remaining: 46s
800:	learn: 41289.6847319	test: 45126.7759583	best: 45126.7759583 (800)	total: 15.9s	remaining: 43.7s
900:	learn: 40600.2447362	test: 44711.4639097	best: 44711.3551914 (899)	total: 17.8s	

In [27]:
y_pred = baseline_model_exp4.predict(X_val)

MAE = round(mean_absolute_error(y_val, y_pred), 2)
R2 = round(r2_score(y_val, y_pred), 4)
MAPE = round(mean_absolute_percentage_error(y_val, y_pred), 4)


print(f"MAE: {MAE}")
print(f"R2-Score: {R2}")
print(f"MAPE: {MAPE}")

MAE: 22560.19
R2-Score: 0.8699
MAPE: 0.1768


### Выводы по Эксперименту 4 — Добавление инфраструктурных признаков

- **MAE:** 22 560 ₽ — ошибка снизилась на ~2.06% по сравнению с предыдущим экспериментом.  
- **R²:** 0.8699 — модель объясняет на ~0.5% больше дисперсии.  
- **MAPE:** 17.68% — относительная ошибка уменьшилась на ~0.7%.

Добавление признаков, отражающих наличие парков, школ, аптек, магазинов и клиник, дало **положительный прирост по всем метрикам**.  
Особенно заметно это на MAE и R², что говорит о том, что инфраструктура действительно учитывается моделью при оценке стоимости аренды.

Признаки инфраструктуры являются **полезными и обоснованными** и должны быть включены в финальный состав признаков модели.
___

## Эксперимент 5 — Повторное обучение модели после удаления выбросов

В этом эксперименте проводится обучение на том же составе признаков, что и в Эксперименте 4, **но после удаления выбросов** из датасета.

Удалены были:
- Объекты с экстремально высокими значениями стоимости аренды (`price_per_month`);
- Квартиры с некорректными или аномальными значениями признаков;
- Записи с неадекватными координатами или расстояниями до метро/центра.

### Цель:
Проверить, как очистка датасета влияет на устойчивость модели и качество предсказания, особенно в массовом сегменте, где выбросы могут искажать обучение.

### Что делается:
- Используется точно такой же состав признаков, как в Эксперименте 4;
- Данные прошли финальную предобработку: удалены выбросы, технические столбцы и шумовые значения;
- Проводится повторное обучение и сравнение метрик с предыдущими экспериментами.

### Гипотеза:
Удаление редких и нетипичных объектов снизит дисперсию и немного улучшит метрики.
___

In [28]:
data = pd.read_csv("cian_rent_Moscow_final.csv")
data.head()

,floor,floors_count,rooms_count,total_meters,price_per_month,district,street,house_number,underground,lat,lon,geo_quality,dist_to_center_km,dist_to_metro_km,parks_1km,groceries_top_1km,schools_1km,pharmacies_top_1km,clinics_1km
0,3,5,1,30.0,30000,Северное Измайлово,Сиреневый бульвар,7,Щелковская,55.802980,37.775445,1,11.387233,1.623468,0,11,4,0,4
1,2,34,0,55.7,25000,Войковский,Адмирала Макарова,6Ак1,Водный стадион,55.833869,37.491056,0,12.054178,0.609563,0,6,1,0,0
2,5,9,1,50.0,30000,Нагорный,Варшавское шоссе,71К1,Варшавская,55.660412,37.620267,1,10.188199,0.788797,0,2,5,0,4
3,1,5,1,32.0,30000,Лосиноостровский,Анадырский проезд,41,Бабушкинская,55.870534,37.693488,1,14.007041,1.818472,0,3,3,1,0
4,10,12,1,23.0,29998,НАО (Новомосковский),Яворки,1к1,Филатов луг,55.509092,37.345487,1,31.956738,10.803737,0,0,1,0,0


In [29]:
X = data.drop("price_per_month", axis = 1)
y = data["price_per_month"]

In [30]:
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, random_state = 42, train_size = 0.8, shuffle = True)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, random_state = 42, test_size = 0.5, shuffle = True)

In [31]:
baseline_model_exp5 = CatBoostRegressor(iterations = 3000,
                                        learning_rate = 0.015,
                                        loss_function="RMSE",
                                        eval_metric="RMSE",
                                        early_stopping_rounds=200,
                                        verbose=100,
                                        task_type="GPU")

In [32]:
baseline_model_exp5.fit(X_train,
                       y_train,
                       eval_set = (X_val, y_val),
                       cat_features = ["district", "street", "underground", "house_number"],
                       use_best_model=True)

0:	learn: 109654.3893040	test: 115561.2005265	best: 115561.2005265 (0)	total: 23.3ms	remaining: 1m 9s
100:	learn: 56768.7294397	test: 56704.7260250	best: 56704.7260250 (100)	total: 2.02s	remaining: 58.1s
200:	learn: 49026.7671982	test: 48064.5710144	best: 48064.5710144 (200)	total: 4.03s	remaining: 56.1s
300:	learn: 46649.0551832	test: 46172.4613759	best: 46172.4613759 (300)	total: 6.07s	remaining: 54.4s
400:	learn: 45229.8577595	test: 45376.6888709	best: 45376.6888709 (400)	total: 8.06s	remaining: 52.2s
500:	learn: 44251.5226684	test: 44805.6200212	best: 44805.6200212 (500)	total: 9.99s	remaining: 49.8s
600:	learn: 43396.3946476	test: 44327.3502238	best: 44327.3502238 (600)	total: 11.9s	remaining: 47.7s
700:	learn: 42559.8566477	test: 43857.1669816	best: 43857.1669816 (700)	total: 13.8s	remaining: 45.2s
800:	learn: 41847.5181466	test: 43451.4065821	best: 43451.4065821 (800)	total: 15.7s	remaining: 43.1s
900:	learn: 41177.5151300	test: 43118.6797238	best: 43118.6797238 (900)	total: 17.

In [33]:
y_pred = baseline_model_exp5.predict(X_val)

MAE = round(mean_absolute_error(y_val, y_pred), 2)
R2 = round(r2_score(y_val, y_pred), 4)
MAPE = round(mean_absolute_percentage_error(y_val, y_pred), 4)


print(f"MAE: {MAE}")
print(f"R2-Score: {R2}")
print(f"MAPE: {MAPE}")

MAE: 22768.53
R2-Score: 0.8841
MAPE: 0.1849


### Выводы по Эксперименту 5 — Повторное обучение после удаления выбросов

- **MAE:** 22 769 ₽ — уменьшение на ~0.93%
- **R²:** 0.8841 — рост на ~1.6%
- **MAPE:** 18.49% — рост на ~4.6%

**Что было сделано**:
Из датасета удалено около **200 объектов** (< 2% выборки), большинство из которых имели **аномально высокие значения расстояния до метро** (например, свыше 100 км), что нехарактерно для Москвы.

**Почему удаление улучшило метрики**:

1. **Устранение шумов и некорректных данных**
   - Объекты с недостоверными координатами (или геопараметрами) искажали пространственные закономерности, мешая обучению модели.

2. **Снижение влияния экстремальных ошибок**
   - Эти точки давали очень высокие остатки, «размазывая» ошибку по всей выборке и ухудшая общую стабильность модели.

3. **Более однородное обучающее пространство**
   - После удаления «вне-Moscow» объектов модель обучалась на более компактном и реалистичном распределении, что позволило ей точнее выучить ключевые зависимости.

**Вывод**:

> Удаление выбросов, даже в небольшом объёме, **может заметно повысить устойчивость и точность модели**, особенно если выбросы связаны с ошибками геокодирования или сильно искажают пространственные признаки.  
> Улучшение R² говорит о том, что модель стала лучше объяснять оставшуюся дисперсию.

___

## Эксперимент 6 — Логарифмирование целевой переменной

В этом эксперименте проводится обучение модели на логарифмированной версии целевой переменной `price_per_month`.

### Цель:
Преобразовать распределение целевой переменной к более нормальному виду, чтобы:
- уменьшить влияние высоких значений на функцию ошибки;
- стабилизировать градиенты обучения;
- повысить устойчивость модели, особенно в среднем и нижнем ценовом сегменте.

### Что делается:
- Значения `price_per_month` преобразуются по формуле `y_log = log1p(y)` — чтобы избежать проблем с нулями;
- Обратное преобразование (`expm1`) применяется к предсказаниям перед оценкой метрик;
- Используются все признаки, отобранные в предыдущем эксперименте (включая координаты, расстояния и инфраструктуру).

### Гипотеза:
Логарифмирование должно снизить среднюю абсолютную ошибку (MAE) и MAPE, а также сгладить переоценку дорогих объектов.
___

In [34]:
baseline_model_exp6 = CatBoostRegressor(iterations = 3000,
                                        learning_rate = 0.015,
                                        loss_function="RMSE",
                                        eval_metric="RMSE",
                                        early_stopping_rounds=200,
                                        verbose=100,
                                        task_type="GPU")

In [35]:
baseline_model_exp6.fit(X_train,
                       np.log1p(y_train),
                       eval_set = (X_val, np.log1p(y_val)),
                       cat_features = ["district", "street", "underground", "house_number"],
                       use_best_model=True)

0:	learn: 0.6637867	test: 0.6705541	best: 0.6705541 (0)	total: 25.3ms	remaining: 1m 15s
100:	learn: 0.3142437	test: 0.3116389	best: 0.3116389 (100)	total: 2.05s	remaining: 58.9s
200:	learn: 0.2616713	test: 0.2597605	best: 0.2597605 (200)	total: 4.08s	remaining: 56.9s
300:	learn: 0.2479701	test: 0.2478806	best: 0.2478806 (300)	total: 6.16s	remaining: 55.2s
400:	learn: 0.2405802	test: 0.2421828	best: 0.2421828 (400)	total: 8.16s	remaining: 52.9s
500:	learn: 0.2362227	test: 0.2387539	best: 0.2387539 (500)	total: 10.1s	remaining: 50.6s
600:	learn: 0.2331671	test: 0.2366091	best: 0.2366091 (600)	total: 12.2s	remaining: 48.5s
700:	learn: 0.2305590	test: 0.2350878	best: 0.2350878 (700)	total: 14.1s	remaining: 46.2s
800:	learn: 0.2284088	test: 0.2337701	best: 0.2337701 (800)	total: 16s	remaining: 44s
900:	learn: 0.2264192	test: 0.2325723	best: 0.2325723 (900)	total: 18s	remaining: 41.9s
1000:	learn: 0.2241129	test: 0.2311985	best: 0.2311985 (1000)	total: 20.1s	remaining: 40.1s
1100:	learn: 0.2

In [36]:
y_pred = np.expm1(baseline_model_exp6.predict(X_val))

MAE = round(mean_absolute_error(y_val, y_pred), 2)
R2 = round(r2_score(y_val, y_pred), 4)
MAPE = round(mean_absolute_percentage_error(y_val, y_pred), 4)


print(f"MAE: {MAE}")
print(f"R2-Score: {R2}")
print(f"MAPE: {MAPE}")

MAE: 22080.65
R2-Score: 0.8827
MAPE: 0.1675


### Выводы по Эксперименту 6 — Логарифмирование целевой переменной

- **MAE:** 22 081 ₽ — ошибка снизилась на ~3.02% по сравнению с предыдущим экспериментом.  
- **R²:** 0.8827 — объяснённая дисперсия снизилась на ~0.15%.  
- **MAPE:** 16.75% — ошибка снизилась на ~9.41% по сравнению с предыдущим экспериментом.

Логарифмирование целевой переменной позволило **снизить абсолютную ошибку (MAE)** — особенно в среднем сегменте цен.  
Однако метрика **R² ухудшилась**, поскольку лог-преобразование сжимает разброс целевой переменной и снижает общую дисперсию, которую модель может объяснить.

Итог: логарифмирование делает модель **более устойчивой к выбросам** и **точнее в массовом сегменте**, но может ухудшить способность различать элитные и редкие квартиры.  
___


## Эксперимент 7 — Увеличением количества итераций и снижение learning rate

В этом эксперименте используется та же логика, что и в Эксперименте 6:  
модель обучается на логарифмированной целевой переменной `log1p(price_per_month)` с использованием всех признаков, включая инфраструктуру, координаты и расстояния.

### Что изменилось:
- **Увеличено количество итераций**: с 3 000 до **15 000**, чтобы дать модели больше времени на сходимость;
- **Уменьшен learning rate**: с 0.015 до **0.00925**, чтобы улучшить устойчивость обучения и позволить модели двигаться к минимуму более плавно.

### Цель:
Проверить, приведёт ли более глубокое и стабильное обучение к снижению MAE и MAPE без потери способности обобщать (R²).

### Гипотеза:
Снижение learning rate и увеличение итераций должны дать модели больше гибкости в обучении и потенциально улучшить качество, особенно в сложных ценовых зонах.
___

In [37]:
baseline_model_exp7 = CatBoostRegressor(iterations = 15000,
                                        learning_rate = 0.00925,
                                        loss_function="RMSE",
                                        eval_metric="RMSE",
                                        early_stopping_rounds=200,
                                        verbose=100,
                                        task_type="GPU")

In [38]:
baseline_model_exp7.fit(X_train,
                       np.log1p(y_train),
                       eval_set = (X_val, np.log1p(y_val)),
                       cat_features = ["district", "street", "underground", "house_number"],
                       use_best_model=True)

0:	learn: 0.6666879	test: 0.6735836	best: 0.6735836 (0)	total: 23.8ms	remaining: 5m 56s
100:	learn: 0.3829040	test: 0.3825513	best: 0.3825513 (100)	total: 2.04s	remaining: 5m 1s
200:	learn: 0.2925882	test: 0.2899941	best: 0.2899941 (200)	total: 4.03s	remaining: 4m 56s
300:	learn: 0.2651218	test: 0.2631590	best: 0.2631590 (300)	total: 6.04s	remaining: 4m 54s
400:	learn: 0.2537113	test: 0.2530892	best: 0.2530892 (400)	total: 8.1s	remaining: 4m 54s
500:	learn: 0.2468630	test: 0.2474793	best: 0.2474793 (500)	total: 10.1s	remaining: 4m 52s
600:	learn: 0.2422545	test: 0.2438284	best: 0.2438284 (600)	total: 12.2s	remaining: 4m 51s
700:	learn: 0.2388075	test: 0.2412566	best: 0.2412566 (700)	total: 14.2s	remaining: 4m 50s
800:	learn: 0.2361914	test: 0.2392354	best: 0.2392354 (800)	total: 16.3s	remaining: 4m 48s
900:	learn: 0.2342213	test: 0.2378560	best: 0.2378560 (900)	total: 18.2s	remaining: 4m 45s
1000:	learn: 0.2323982	test: 0.2366196	best: 0.2366196 (1000)	total: 20.2s	remaining: 4m 42s
11

In [39]:
y_pred = np.expm1(baseline_model_exp7.predict(X_val))

MAE = round(mean_absolute_error(y_val, y_pred), 2)
R2 = round(r2_score(y_val, y_pred), 4)
MAPE = round(mean_absolute_percentage_error(y_val, y_pred), 4)


print(f"MAE: {MAE}")
print(f"R2-Score: {R2}")
print(f"MAPE: {MAPE}")

MAE: 21287.23
R2-Score: 0.8916
MAPE: 0.1631


### Выводы по Эксперименту 7 — Логарифмирование + увеличение итераций + снижение learning rate

- **MAE:** 21 287 ₽ — ошибка снизилась на ~3.6% по сравнению с предыдущим логарифмированным экспериментом.  
- **R²:** 0.8916 — объяснённая дисперсия выросла на ~1%.  
- **MAPE:** 16.31% — относительная ошибка снизилась на ~2.6%.

Увеличение количества итераций до 15 000 и снижение learning rate до 0.00925 позволило модели **обучиться глубже и стабильнее**, особенно в среднеценовом сегменте.  
Все три метрики улучшились, включая R², что говорит об улучшении общей способности модели объяснять разброс в данных.

Вывод: сочетание логарифмирования, увеличенного количества итераций и малого learning rate является **сильной основой** для дальнейшего подбора гиперпараметров (Optuna).

___

## Эксперимент 8 — Подбор гиперпараметров с Optuna

В этом эксперименте проводится автоматизированный подбор гиперпараметров модели `CatBoostRegressor` с использованием библиотеки **Optuna**.

### Базовая конфигурация:
- Целевая переменная: `log1p(price_per_month)`  
- Количество итераций: **15 000**
- Learning rate: **0.00925**
- Валидация: 20% hold-out (как и в предыдущих экспериментах)
- Оценка качества: **RMSE** на логарифмированной шкале → предсказания преобразуются обратно через `expm1()` для финальной оценки метрик

### Подбираются параметры:
- `depth` — глубина деревьев
- `l2_leaf_reg` — L2-регуляризация
- `min_data_in_leaf` — минимальное число объектов в листе
- `random_strength` — шум на сплите
- `bootstrap_type` - метод создания выборки
- `bagging_temperature` — параметр стохастичности

### Цель:
Найти оптимальное сочетание гиперпараметров, которое обеспечит минимальную ошибку на валидационной выборке, повысив точность предсказаний модели.

### Гипотеза:
Подбор параметров на логарифмированных данных с глубоким обучением позволит достичь **наилучшего баланса между переобучением и недообучением**, снизив MAE и RMSE на обратной шкале.
___

In [40]:
def objective(trial):
    params = {
        "depth": trial.suggest_int("depth", 6, 10),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-2, 100.0, log=True),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 20),
        "random_strength": trial.suggest_float("random_strength", 1e-4, 10.0, log=True),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),
        "eval_metric": "RMSE",
        "loss_function": "RMSE",
        "early_stopping_rounds": 200,
        "task_type": "GPU",
        "verbose": 0
        }

    if params["bootstrap_type"] == "Bayesian":
        params["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0.0, 10.0, step = 0.01)

    if params["bootstrap_type"] in ["Bernoulli", "MVS"]:
        params["subsample"] = trial.suggest_float("subsample", 0.6, 1.0, step = 0.01)

    model = CatBoostRegressor(**params,
                             iterations = 15_000,
                             learning_rate = 0.00925)


    model.fit(
        X_train,
        np.log1p(y_train),
        eval_set=(X_val, np.log1p(y_val)),
        cat_features=["district", "street", "underground", "house_number"],
        use_best_model=True
    )

    y_pred = model.predict(X_val)

    return root_mean_squared_error(np.log1p(y_val), y_pred)

In [41]:
study = optuna.create_study(study_name = "CB1", direction="minimize")

[I 2025-05-25 20:05:16,978] A new study created in memory with name: CB1


In [42]:
study.optimize(objective, n_trials=40, show_progress_bar = True)

Best trial: 0. Best value: 0.2217:   2%|▎         | 1/40 [11:50<7:42:06, 710.94s/it]

[I 2025-05-25 20:17:07,951] Trial 0 finished with value: 0.22170014197065774 and parameters: {'depth': 10, 'l2_leaf_reg': 27.173594647333527, 'min_data_in_leaf': 13, 'random_strength': 0.008640596017304658, 'bootstrap_type': 'MVS', 'subsample': 0.96}. Best is trial 0 with value: 0.22170014197065774.


Best trial: 1. Best value: 0.212031:   5%|▌         | 2/40 [16:54<4:58:23, 471.13s/it]

[I 2025-05-25 20:22:11,217] Trial 1 finished with value: 0.21203110744388215 and parameters: {'depth': 6, 'l2_leaf_reg': 0.027898142543929954, 'min_data_in_leaf': 20, 'random_strength': 0.011666534962321888, 'bootstrap_type': 'Bernoulli', 'subsample': 0.61}. Best is trial 1 with value: 0.21203110744388215.


Best trial: 1. Best value: 0.212031:   8%|▊         | 3/40 [23:55<4:36:34, 448.51s/it]

...

[I 2025-05-25 23:42:10,494] Trial 37 finished with value: 0.21896209166207986 and parameters: {'depth': 6, 'l2_leaf_reg': 0.17137661016759878, 'min_data_in_leaf': 20, 'random_strength': 0.0007934959387589931, 'bootstrap_type': 'MVS', 'subsample': 0.8}. Best is trial 9 with value: 0.2106117073181563.


Best trial: 9. Best value: 0.210612:  98%|█████████▊| 39/40 [3:45:23<05:26, 326.51s/it]

[I 2025-05-25 23:50:40,986] Trial 38 finished with value: 0.2176887013712411 and parameters: {'depth': 8, 'l2_leaf_reg': 0.3590127943534598, 'min_data_in_leaf': 11, 'random_strength': 0.00011198585851737312, 'bootstrap_type': 'Bayesian', 'bagging_temperature': 5.37}. Best is trial 9 with value: 0.2106117073181563.


Best trial: 9. Best value: 0.210612: 100%|██████████| 40/40 [3:52:20<00:00, 348.51s/it]

[I 2025-05-25 23:57:37,551] Trial 39 finished with value: 0.2115396072071057 and parameters: {'depth': 9, 'l2_leaf_reg': 0.027119378549036732, 'min_data_in_leaf': 14, 'random_strength': 0.023204087220495193, 'bootstrap_type': 'Bernoulli', 'subsample': 0.6}. Best is trial 9 with value: 0.2106117073181563.


In [43]:
print(f"Best Params: {study.best_params}")
print(f"Study Best Score: {study.best_value}")

Best Params: {'depth': 7, 'l2_leaf_reg': 0.49704978003330685, 'min_data_in_leaf': 17, 'random_strength': 0.0029363237257199605, 'bootstrap_type': 'Bernoulli', 'subsample': 0.76}
Study Best Score: 0.2106117073181563


In [44]:
params = study.best_params

In [45]:
baseline_model_exp8 = CatBoostRegressor(**params,
                                        iterations = 15000,
                                        learning_rate = 0.00925,
                                        loss_function="RMSE",
                                        eval_metric="RMSE",
                                        early_stopping_rounds=200,
                                        verbose=100,
                                        task_type="GPU")

In [46]:
baseline_model_exp8.fit(
    X_train,
    np.log1p(y_train),
    eval_set=(X_val, np.log1p(y_val)),
    cat_features=["district", "street", "underground", "house_number"],
    use_best_model=True)

0:	learn: 0.6665150	test: 0.6734339	best: 0.6734339 (0)	total: 27.1ms	remaining: 6m 45s
100:	learn: 0.3733375	test: 0.3736676	best: 0.3736676 (100)	total: 2.58s	remaining: 6m 21s
200:	learn: 0.2838050	test: 0.2831082	best: 0.2831082 (200)	total: 5.23s	remaining: 6m 24s
300:	learn: 0.2568795	test: 0.2579486	best: 0.2579486 (300)	total: 7.89s	remaining: 6m 25s
400:	learn: 0.2438804	test: 0.2470458	best: 0.2470458 (400)	total: 10.6s	remaining: 6m 24s
500:	learn: 0.2364393	test: 0.2413405	best: 0.2413405 (500)	total: 13.2s	remaining: 6m 21s
600:	learn: 0.2312835	test: 0.2378533	best: 0.2378533 (600)	total: 15.8s	remaining: 6m 17s
700:	learn: 0.2276004	test: 0.2354961	best: 0.2354961 (700)	total: 18.3s	remaining: 6m 12s
800:	learn: 0.2249802	test: 0.2338367	best: 0.2338366 (799)	total: 20.6s	remaining: 6m 4s
900:	learn: 0.2232509	test: 0.2327369	best: 0.2327369 (900)	total: 22.7s	remaining: 5m 55s
1000:	learn: 0.2217708	test: 0.2317262	best: 0.2317262 (1000)	total: 24.7s	remaining: 5m 44s
1

In [41]:
y_pred = np.expm1(baseline_model_exp8.predict(X_val))

MAE = round(mean_absolute_error(y_val, y_pred), 2)
R2 = round(r2_score(y_val, y_pred), 4)
MAPE = round(mean_absolute_percentage_error(y_pred, y_val), 4)


print(f"MAE: {MAE}")
print(f"R2-Score: {R2}")
print(f"MAPE: {MAPE}")

MAE: 20388.55
R2-Score: 0.9
MAPE: 0.1528


### Выводы по Эксперименту 8 — Подбор гиперпараметров 

- **MAE:** 20 388 ₽ — абсолютная ошибка снизилась на ~4.2% по сравнению с предыдущим этапом.  
- **R²:** 0.9 — объяснённая дисперсия увеличилась на ~0.94%.  
- **MAPE:** 15.28% — относительная ошибка снизилась на ~6.3%.

Автоматизированный подбор гиперпараметров с помощью Optuna привёл к улучшению всех ключевых метрик.  
Обучение стало более стабильным, модель точнее описывает структуру данных и показывает более уверенные предсказания.  
Текущая конфигурация служит прочной основой для последующего анализа ошибок и внедрения bias correction.

Вывод: модель после тюнинга с помощью Optuna достигла лушегого качества.
___

In [158]:
joblib.dump(baseline_model_exp8, "final_rent_model_catboost.pkl")
print("Финальная модель сохранена как 'final_rent_model_catboost.pkl'")

Финальная модель сохранена как 'final_rent_model_catboost.pkl'


In [161]:
model_params = baseline_model_exp8.get_params()

with open("final_model_params.json", "w", encoding="utf-8") as f:
    json.dump(model_params, f, indent=4, ensure_ascii=False)

print("Параметры модели сохранены в 'final_model_params.json'")

Параметры модели сохранены в 'final_model_params.json'


## Финальное сравнение: Бейзлайн vs Оптимизированная модель CatBoost

| Метрика     | Базовый бейзлайн        | Финальная модель (Optuna) | Изменение            |
|-------------|--------------------------|----------------------------|------------------------|
| **MAE**     | 24 814 ₽                | **20 388 ₽**              | **−4 426 ₽** (−17.8%)  
| **R²**      | 0.8461                  | **0.9**                  | **+0.0539** (+6.4%)  
| **MAPE**    | 18.81%                  | **15.28%**                | **−3.53 п.п.** (−18.7%)

---

### Интерпретация:

- **MAE снизилась почти на 4.5 тыс. ₽**, что делает модель заметно более точной в денежном выражении.
- **R² вырос до 0.9**, что означает: модель объясняет **почти 90% дисперсии цен** — по сравнению с 84.61% в бейзлайне.
- **MAPE уменьшилась более чем на 3.53 процентных пункта**, что делает предсказания более стабильными и приемлемыми для практического использования.

---

### Вывод:

> Модель после подбора гиперпараметров с помощью **Optuna** продемонстрировала **устойчивый и значимый рост качества** по всем ключевым метрикам.  
___